<a href="https://colab.research.google.com/github/LeonardoViotti/cdr-training/blob/develop/notebooks/aggregated-cdr-analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CDR Analysis

 - Aggregated data
 - Quality Checks
 - Analysis
 - Map

In [ ]:
!pip install geopandas

In [4]:
import pandas as pd
import numpy as np
import geopandas as gpd

## Exercise 1

## Exercise 2


## Exercise 3